In [1]:
import json
import os
import os.path as osp
from glob import glob
import re
import copy
import warnings
import argparse

import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from sklearn.impute import KNNImputer

# warnings.filterwarnings(action='ignore')

In [2]:
date = "20211218" #divided the file by date
json_file_path = "D:/Dataset_BENG/main_labels" #main JSON file path from AIDK
video_file_path= "D:/Dataset_BENG/main_videos" #main video file path from AIDK
tol = 45 #nearest N frames

In [3]:
anno_file_list=glob(osp.join(json_file_path, date, '*'))
anno_file_list

['D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam1_20211218_073000.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam1_20211218_125500.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam1_20211218_180000.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam1_20211218_230000.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam2_20211218_073226.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam2_20211218_125722.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam2_20211218_180236.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam2_20211218_230230.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam3_20211218_073420.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam3_20211218_130004.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam3_20211218_180511.json',
 'D:/Dataset_BENG/main_labels\\20211218\\pig_hampyeong_cam3_20211218_230459.json',
 'D:

In [ ]:
# manually key in the file index and run the code
json_path = anno_file_list[6]

with open(json_path, 'r') as f:
    json_file=json.load(f)

In [ ]:
# frame_num=len(json_file['label_info']['annotations'])

In [ ]:
# create a list of pig IDs created in the 1-second long episode
all_object_ids = []
for frame in json_file['label_info']['annotations']:
    for obj in frame:
        id_ = obj['object_id']
        if id_ not in all_object_ids:
            all_object_ids.append(id_)

In [ ]:
'''
dictionary storing the BBOX coordinates corresponding to each pig ID throughout the whole episode
'''

dict_frame={'obj_ids':[], 'bbox_dict': {}}

for frame in tqdm(json_file['label_info']['annotations']):

    obj_in_current_frame = []

    for obj in frame:
        id_ = obj['object_id']
        bbox=obj['bbox']

        obj_in_current_frame.append(id_)

        if id_ not in dict_frame['obj_ids']:
            dict_frame['obj_ids'].append(id_)
            dict_frame['obj_ids'].sort()    
            # make_bbox_dict
            dict_frame['bbox_dict'][str(id_)] = {str(key): [] for key in range(4)}

        for i, coord in enumerate(bbox):
            dict_frame['bbox_dict'][str(id_)][str(i)].append(coord)

    for id_ in all_object_ids:
        # it is not in current frame, and this is its first appearance
        if (id_ not in obj_in_current_frame) & (id_ not in dict_frame['obj_ids']):
            dict_frame['obj_ids'].append(id_)
            dict_frame['bbox_dict'][str(id_)] = {str(key): [0] for key in range(4)}
        # it is not in current frame, but it has appeared in previous frames
        elif (id_ not in obj_in_current_frame) & (id_ in dict_frame['obj_ids']):
            dict_frame['bbox_dict'][str(id_)][str(0)].append(0)
            dict_frame['bbox_dict'][str(id_)][str(1)].append(0)
            dict_frame['bbox_dict'][str(id_)][str(2)].append(0)
            dict_frame['bbox_dict'][str(id_)][str(3)].append(0)

In [ ]:
'''
check if coord info is given for each bbox point
'''
for ids in all_object_ids:
    for j in range(4):
        if len(dict_frame['bbox_dict'][str(ids)][str(j)]) != 1800:
            print("ID", ids, "does not have 1800 frames at point", j)

# Imputation methods

### Miscellaneous methods

#### KNN Imputer

In [ ]:
smooth_dict_frame=copy.deepcopy(dict_frame)
imputer = KNNImputer(n_neighbors=5, weights='distance', metric='nan_euclidean')
        
for id_ in smooth_dict_frame['bbox_dict']:

    edge_pts_df = pd.DataFrame(data = (smooth_dict_frame['bbox_dict'][id_]['0'], smooth_dict_frame['bbox_dict'][id_]['1'],
                                      smooth_dict_frame['bbox_dict'][id_]['2'], smooth_dict_frame['bbox_dict'][id_]['3'])).T
    imputer.fit(edge_pts_df)
    imputed_df = imputer.transform(edge_pts_df)
    imputed_df = imputed_df.astype(int).T
    for j, each_edge in enumerate(smooth_dict_frame['bbox_dict'][id_]):
        imputed_seq = imputed_df[j]
        smooth_dict_frame['bbox_dict'][id_][each_edge]=imputed_seq        
        

#### Polynomial regression

In [ ]:
smooth_dict_frame=copy.deepcopy(dict_frame)

for id_ in smooth_dict_frame['bbox_dict']:

    for each_edge in smooth_dict_frame['bbox_dict'][id_]:
        edge_pts_list = smooth_dict_frame['bbox_dict'][id_][each_edge]
        x=list(range(1, len(edge_pts_list)+1))
        fitted_model=np.poly1d(np.polyfit(x, edge_pts_list, deg=poly_degree))
        fitted_seq=fitted_model(x)
        fitted_seq=fitted_seq.astype(np.int)
        
        smooth_dict_frame['bbox_dict'][id_][each_edge]=fitted_seq


#### Moving Average

In [ ]:
smooth_dict_frame=copy.deepcopy(dict_frame)
for id_ in smooth_dict_frame['bbox_dict']:

    for each_edge in smooth_dict_frame['bbox_dict'][id_]:
        bbox_seq=smooth_dict_frame['bbox_dict'][id_][each_edge]

        fitted_seq=pd.Series(bbox_seq).rolling(window=wins, min_periods=1).mean().astype(int).values
#         fitted_seq=fitted_seq.fillna(method='backfill').astype(np.int).values

        smooth_bbox_seq=list(fitted_seq)
        smooth_dict_frame['bbox_dict'][id_][each_edge]=smooth_bbox_seq

#### Forward fill method

In [ ]:
smooth_dict_frame=copy.deepcopy(dict_frame)

for id_ in smooth_dict_frame['bbox_dict']:

    for each_edge in smooth_dict_frame['bbox_dict'][id_]:
        
        edge_pts_list = smooth_dict_frame['bbox_dict'][id_][each_edge]
        edge_pts_list = pd.Series(edge_pts_list)
        edge_pts_list = list(edge_pts_list.fillna(method='ffill').fillna(0).astype(int))
                    
        smooth_dict_frame['bbox_dict'][id_][each_edge]=edge_pts_list

### Final method chosen for thesis

#### Simply copy the value of nearest "non-empty" frame within distance of N (set as 'tol' above)

In [ ]:
smooth_dict_frame=copy.deepcopy(dict_frame)

for id_ in smooth_dict_frame['bbox_dict']:

    for each_edge in smooth_dict_frame['bbox_dict'][id_]:
        
        edge_pts_list = smooth_dict_frame['bbox_dict'][id_][each_edge]

        for i, point in enumerate(edge_pts_list):
            if point == 0:
                if (sum(edge_pts_list[max(0, i-tol):i]) > 0) & (sum(edge_pts_list[i+1:i+tol+1]) > 0):
                    try:
                        edge_pts_list[i] = edge_pts_list[i-1]
                    except:
                        continue
                    
        smooth_dict_frame['bbox_dict'][id_][each_edge]=edge_pts_list


In [ ]:
json_copy=copy.deepcopy(json_file)

for f_idx, frame in enumerate(json_copy['label_info']['annotations']):
    
    obj_in_current_frame = []    
    for obj in frame:
        # check new id
        id_=obj['object_id']
        obj_in_current_frame.append(id_)
        
        smooth_bbox=[]
        for edge in range(4):
            smooth_bbox.append(smooth_dict_frame['bbox_dict'][str(id_)][str(edge)][f_idx])                

        # get object index and replace bbox annotation
        k=obj_in_current_frame.index(id_)
        json_copy['label_info']['annotations'][f_idx][k]['bbox']=smooth_bbox
            
    for id_ in dict_frame['obj_ids']:
        if id_ not in obj_in_current_frame:
            smooth_bbox=[]
            for edge in range(4):
                smooth_bbox.append(smooth_dict_frame['bbox_dict'][str(id_)][str(edge)][f_idx])    
            missing_dict = {'bbox': smooth_bbox, 'frame_num': f_idx+1, 'object_id': id_} #impute
            frame.append(missing_dict)

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [ ]:
'''
save new imputed bbox
'''
if not osp.exists(osp.join(json_file_path, 'tol45', date)):
    os.makedirs(osp.join(json_file_path, 'tol45', date))

with open(json_path.replace('main_labels', 'main_labels/tol45'), 'w') as outfile:
    json.dump(json_copy, outfile, cls=NpEncoder)

<br>
<br>

### SAVE the video with display of pigID and BBOX
*CAN IGNORE THIS! just for visualization*

In [ ]:
save_video = True
resize_rate = 1/2
frame_limit= 60*30 ## 60sec, 30FPS
def main():
    np.random.seed(32)
    colours = np.random.randint(10, 245, size=(64, 3))  ## Used only for display

    
    idx= 0
    
    while True:
        frame_count= 0
        video_file= 'pig_hampyeong_cam4_20211218_073523_imputed.avi'
#         video_file= new_video_list[idx]
        ################################
        ## Check Video file and Open
#         video_file_path = os.path.join(video_path, video_file)
#         video_file_path = 'D:/Dataset_BENG/main_videos/20211217/pig_hampyeong_cam3_20211217_073449.avi'      
        video_file_path = video_path
        capture = cv2.VideoCapture(video_file_path)

        if not capture.isOpened():
            print("\n Video file does not exist. (%s)"% video_path)
            continue
        
        print("Video Open: %s"% video_file_path)
        video_fps= int(capture.get(cv2.CAP_PROP_FPS))

        ################################
        ## Check JSON file and Open
#         label_file_path= 'D:/Dataset_BENG/main_labels/tol45/20211216/pig_hampyeong_cam3_20211216_230507.json'
        label_file_path = label_path
        
        if os.path.exists(label_file_path)== False:
            print("\n json file does not exist. (%s.json)"% os.path.splitext(video_file)[0])
            continue

        with open(label_file_path, "r") as jf:
            json_data = json.load(jf)
            
        ################################
        ## For Save video
        if save_video:
            video_width= int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            video_height= int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))       
            fourcc = cv2.VideoWriter_fourcc(*'DIVX')
            out = cv2.VideoWriter(video_file, fourcc, video_fps, (video_width, video_height))

        ################################
        ## Show Images
        while capture.isOpened():
            res, img = capture.read()
            current_frame= int(capture.get(cv2.CAP_PROP_POS_FRAMES))            

            if img is None:
                idx= (idx+1)%(len(video_list))
                print("Last frame")
                break

            frame =json_data['label_info']['annotations'][current_frame-1]
            for annotation in frame:
                bbox= annotation['bbox']
                object_id= annotation['object_id']
                # derive the centre of the bounding box
                centre_x = int((bbox[0] + bbox[2])/2)
                centre_y = int((bbox[1] + bbox[3])/2)

                ## Draw Rectangle (BBox)
                cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                            (int(colours[object_id % 64, 0]), int(colours[object_id % 64, 1]), int(colours[object_id % 64, 2])), 1)
#                 cv2.circle(img, (centre_x, centre_y), radius=5, color=(255, 255, 0), thickness=-1)

                ## Print Pig ID
                cv2.putText(img, "%d"% object_id, (bbox[0], bbox[1]+25), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                    (255, 255, 255), 10)
                cv2.putText(img, "%d"% object_id, (bbox[0], bbox[1]+25), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (int(colours[object_id % 64, 0]), int(colours[object_id % 64, 1]), int(colours[object_id % 64, 2])), 2)

            #############################
            ## For information Text
            video_file = video_path.split('/')[-1]
            cv2.putText(img, video_file, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
            cv2.putText(img, str(current_frame), (1000, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
#             cv2.putText(img, "A:Prev / D:Next", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
#             cv2.putText(img, "Q:Exit", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
            
            if save_video:
                out.write(img)
                
            frame_count= frame_count+ 1

            cv2.imshow("Tracking result", cv2.resize(img, (0,0), fx=resize_rate, fy= resize_rate))
            key = cv2.waitKey(video_fps)
#             key = cv2.waitKey(1)

            ## exit Q key
            if key == ord('q'):
                capture.release()
                exit()
            ## next A key
#             if key == ord('a'):
#                 idx= (idx-1)%(len(video_list))
#                 break
#             if key == ord('d') or frame_count >= frame_limit:
#                 idx= (idx+1)%(len(video_list))
#                 break
            if key == ord('p'):
                print("playful behavior appears at frame #", current_frame)
                cv2.waitKey(10000)
        
        capture.release()



if __name__ == "__main__":
    main()

Video Open: D:/Dataset_BENG/main_videos/20211218/pig_hampyeong_cam4_20211218_073523.avi
